In [461]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [462]:
deliveries_df = pd.read_csv('deliveries.csv')

In [463]:
deliveries_df

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260915,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,AK Markram,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260916,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,AK Markram,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260917,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260918,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN


In [464]:
deliveries_df.isna().sum()

# this is fine, we aren't using those columns

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batter                   0
bowler                   0
non_striker              0
batsman_runs             0
extra_runs               0
total_runs               0
extras_type         246795
is_wicket                0
player_dismissed    247970
dismissal_kind      247970
fielder             251566
dtype: int64

## Drop batsmen who have not played much

In [465]:
batter_freq = deliveries_df.batter.value_counts().reset_index()
batter_freq

,batter,count
0,V Kohli,6236
1,S Dhawan,5483
2,RG Sharma,5183
3,DA Warner,4849
4,SK Raina,4177
...,...,...
668,V Pratap Singh,1
669,Sunny Gupta,1
670,YA Abdulla,1
671,SZ Mulani,1


In [466]:
batsmen_to_drop = batter_freq[batter_freq['count'] < 18]
# they have batted against less than 18 balls in their whole career

In [467]:
batsmen_to_drop

,batter,count
505,JJ van der Wath,17
506,X Thalaivan Sargunam,17
507,VG Arora,17
508,SMSM Senanayake,17
509,Karanveer Singh,17
...,...,...
668,V Pratap Singh,1
669,Sunny Gupta,1
670,YA Abdulla,1
671,SZ Mulani,1


In [468]:
list_of_batsmen_to_drop = batsmen_to_drop.batter.to_list()
list_of_batsmen_to_drop

['JJ van der Wath',
 'X Thalaivan Sargunam',
 'VG Arora',
 'SMSM Senanayake',
 'Karanveer Singh',
 'R Bishnoi',
 'AU Rashid',
 'Saurav Chauhan',
 'Yudhvir Singh',
 'T Henderson',
 'Kartik Tyagi',
 'Umran Malik',
 'SB Joshi',
 'GR Napier',
 'TU Deshpande',
 'Mujeeb Ur Rahman',
 'CJ McKay',
 'JE Root',
 'UA Birla',
 'Gulbadin Naib',
 'ER Dwivedi',
 'TS Mills',
 'RA Bawa',
 'BB Sran',
 'SM Harwood',
 'P Awana',
 'AG Murtaza',
 'B Laughlin',
 'D du Preez',
 'MDKJ Perera',
 'Fazalhaq Farooqi',
 'Vijaykumar Vyshak',
 'F Behardien',
 'P Sahu',
 'DP Nannes',
 'SD Lad',
 'K Kartikeya',
 'J Theron',
 'DT Patil',
 'ND Doshi',
 'JL Pattinson',
 'Pankaj Singh',
 'AR Bawne',
 'DG Nalkande',
 'Harmeet Singh',
 'AA Noffke',
 'Akash Deep',
 'Mohammad Ashraful',
 'PSP Handscomb',
 'TM Srivastava',
 'BJ Haddin',
 'Jaskaran Singh',
 'AA Chavan',
 'I Udana',
 'D Ferreira',
 'AJ Hosein',
 'A Singh',
 'S Ladda',
 'MA Wood',
 'CK Langeveldt',
 'KAJ Roach',
 'DAJ Bracewell',
 'JP Behrendorff',
 'Ankit Soni',
 

In [469]:
deliveries_df = deliveries_df[~deliveries_df['batter'].isin(list_of_batsmen_to_drop)]

In [470]:
deliveries_df

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260915,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,AK Markram,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260916,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,AK Markram,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260917,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260918,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN


## Drop bowlers who have not played much

In [471]:
bowler_freq = deliveries_df.bowler.value_counts().reset_index()

In [472]:
list_of_bowlers_to_drop = bowler_freq[bowler_freq['count'] < 18].bowler.to_list()
list_of_bowlers_to_drop

['CJ Green',
 'CK Kapugedera',
 'Monu Kumar',
 'S Rana',
 'R Sanjay Yadav',
 'Abdur Razzak',
 'LRPL Taylor',
 'AA Kazi',
 'Arshad Khan (2)',
 'S Vidyut',
 'S Midhun',
 'MJ Suthar',
 'FY Fazal',
 'M Shahrukh Khan',
 'Mandeep Singh',
 'JE Root',
 'RR Bhatkal',
 'Mohit Rathee',
 'N Wadhera',
 'R Ravindra',
 'RR Bose',
 'C Munro',
 'Sachin Baby',
 'NA Saini',
 'LA Carseldine',
 'BJ Rohrer',
 'SS Mundhe',
 'Gulbadin Naib',
 'LPC Silva',
 'SS Iyer',
 'Aman Hakim Khan',
 'AM Rahane',
 'RA Shaikh',
 'RS Gavaskar',
 'C Ganapathy',
 'I Malhotra',
 'B Chipli',
 'P Prasanth',
 'Y Gnaneswara Rao',
 'F du Plessis',
 'SA Yadav',
 'RA Tripathi',
 'D Brevis',
 'SPD Smith',
 'SN Khan',
 'DA Warner',
 'AC Gilchrist',
 'YBK Jaiswal',
 'Atharva Taide']

In [473]:
deliveries_df = deliveries_df[~deliveries_df['bowler'].isin(list_of_bowlers_to_drop)]
deliveries_df

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260915,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,AK Markram,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260916,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,AK Markram,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260917,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260918,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN


## Career Bowling average

In [474]:
bowling = deliveries_df.groupby(by=['bowler']).agg({'total_runs':'sum', 'is_wicket':'sum'}).reset_index()

In [475]:
# First, let's create a mask for valid dismissals (wickets that count for bowling average).
# I only need to use the dismissals that are valid for calculating bowling average. So, I'll first create a boolean array depicting which dismissals are valid
valid_dismissals = ~deliveries_df['dismissal_kind'].isin(['run out', 'retired hurt', 'retired out', 'obstructing the field'])

# Now calculate the bowling statistics with this condition
bowling = deliveries_df.groupby(by=['bowler']).agg({
    'total_runs': 'sum',
    # Only count wickets where is_wicket=1 AND dismissal_kind is valid. We use '&', not 'and' since we want to perform element wise "AND" operation across the two pandas series (x == 1) and (valid_dismissals[x.index])
    'is_wicket': lambda x: ((x == 1) & (valid_dismissals[x.index])).sum()
}).reset_index()
bowling

,bowler,total_runs,is_wicket
0,A Ashish Reddy,399,18
1,A Badoni,37,2
2,A Chandila,245,11
3,A Choudhary,144,5
4,A Dananjaya,47,0
...,...,...,...
476,Yash Dayal,932,28
477,Yash Thakur,700,24
478,Yudhvir Singh,125,4
479,Yuvraj Singh,1091,36


In [476]:
bowling['career_bowling_average'] = bowling['total_runs']/bowling['is_wicket']

In [477]:
bowling['career_bowling_average']= bowling['career_bowling_average'].replace(np.inf, -np.inf)
bowling['career_bowling_average']= bowling['career_bowling_average'].replace(-np.inf, bowling['career_bowling_average'].max())


In [478]:
bowling

,bowler,total_runs,is_wicket,career_bowling_average
0,A Ashish Reddy,399,18,22.166667
1,A Badoni,37,2,18.500000
2,A Chandila,245,11,22.272727
3,A Choudhary,144,5,28.800000
4,A Dananjaya,47,0,137.000000
...,...,...,...,...
476,Yash Dayal,932,28,33.285714
477,Yash Thakur,700,24,29.166667
478,Yudhvir Singh,125,4,31.250000
479,Yuvraj Singh,1091,36,30.305556


In [479]:
bowling[bowling['career_bowling_average'] == bowling['career_bowling_average'].min()]

,bowler,total_runs,is_wicket,career_bowling_average
434,T Stubbs,29,4,7.25


In [480]:
bowling['career_bowling_average'].describe()

count    481.000000
mean      44.583180
std       31.742902
min        7.250000
25%       27.500000
50%       33.225806
75%       45.250000
max      137.000000
Name: career_bowling_average, dtype: float64

## Career Batting average

In [481]:
batting = deliveries_df.groupby(by=['batter']).agg({'batsman_runs':'sum', 'is_wicket':'sum'}).reset_index()
batting['career_batting_average'] = batting['batsman_runs']/batting['is_wicket']

In [482]:
batting['career_batting_average'].describe()

/opt/anaconda3/envs/optiProj_env/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    505.000000
mean            inf
std             NaN
min        1.600000
25%       11.000000
50%       18.119048
75%       25.400000
max             inf
Name: career_batting_average, dtype: float64

In [483]:
batting['career_batting_average']= batting['career_batting_average'].replace(np.inf, -np.inf)
batting['career_batting_average']= batting['career_batting_average'].replace(-np.inf, batting['career_batting_average'].max())


In [484]:
batting['career_batting_average'].describe()

count    505.000000
mean      19.392765
std       10.968157
min        1.600000
25%       11.000000
50%       18.119048
75%       25.400000
max       88.000000
Name: career_batting_average, dtype: float64

## Performance index for batsmen

In [485]:
df = deliveries_df.groupby(by=['batter', 'bowler']).agg({'batsman_runs':'sum', 'is_wicket':'sum'}).reset_index()

In [486]:
df.dropna(inplace=True)

In [487]:
df

,batter,bowler,batsman_runs,is_wicket
0,A Ashish Reddy,A Nehra,7,1
1,A Ashish Reddy,AB Dinda,9,0
2,A Ashish Reddy,AD Mathews,25,0
3,A Ashish Reddy,AD Russell,4,1
4,A Ashish Reddy,Anureet Singh,2,0
...,...,...,...,...
27232,Z Khan,SR Watson,3,0
27233,Z Khan,Shakib Al Hasan,3,0
27234,Z Khan,Sohail Tanvir,3,0
27235,Z Khan,VRV Singh,0,0


In [488]:
df.is_wicket.value_counts()

is_wicket
0    17328
1     7881
2     1471
3      376
4      129
5       29
6       13
7        9
9        1
Name: count, dtype: int64

In [489]:
df['batting_average'] = df['batsman_runs']/df['is_wicket']

In [490]:
df['batting_average']

0        7.0
1        inf
2        inf
3        4.0
4        inf
        ... 
27232    inf
27233    inf
27234    inf
27235    NaN
27236    inf
Name: batting_average, Length: 27237, dtype: float64

In [491]:
df.dropna(inplace=True) #Removing null values as a precautionary value

In [492]:
temp_df = df[~np.isinf(df['batting_average'])]

In [493]:
best_per_batsman = temp_df.groupby(by=['batter']).agg({'batting_average':'mean'})

In [494]:
best_per_batsman

,batting_average
batter,
A Ashish Reddy,4.500000
A Badoni,8.454545
A Chopra,2.800000
A Choudhary,4.000000
A Flintoff,2.500000
...,...
YV Dhull,3.333333
YV Takawale,3.625000
Yashpal Singh,4.500000


In [495]:
mapping = best_per_batsman['batting_average'].to_dict()

In [496]:
df['batting_average'] = np.where(
    df['batting_average'] == np.inf,
    df['batter'].map(mapping),
    df['batting_average']
)


In [497]:
df.head(5)

,batter,bowler,batsman_runs,is_wicket,batting_average
0,A Ashish Reddy,A Nehra,7,1,7.0
1,A Ashish Reddy,AB Dinda,9,0,4.5
2,A Ashish Reddy,AD Mathews,25,0,4.5
3,A Ashish Reddy,AD Russell,4,1,4.0
4,A Ashish Reddy,Anureet Singh,2,0,4.5


In [498]:
df.dropna(inplace=True)

In [499]:
df

,batter,bowler,batsman_runs,is_wicket,batting_average
0,A Ashish Reddy,A Nehra,7,1,7.000000
1,A Ashish Reddy,AB Dinda,9,0,4.500000
2,A Ashish Reddy,AD Mathews,25,0,4.500000
3,A Ashish Reddy,AD Russell,4,1,4.000000
4,A Ashish Reddy,Anureet Singh,2,0,4.500000
...,...,...,...,...,...
27231,Z Khan,SL Malinga,1,0,1.769231
27232,Z Khan,SR Watson,3,0,1.769231
27233,Z Khan,Shakib Al Hasan,3,0,1.769231
27234,Z Khan,Sohail Tanvir,3,0,1.769231


In [500]:
mapping = dict(zip(bowling['bowler'], bowling['career_bowling_average']))
df['career_bowling_average'] = df['bowler'].map(mapping)

In [501]:
df.head(5)

,batter,bowler,batsman_runs,is_wicket,batting_average,career_bowling_average
0,A Ashish Reddy,A Nehra,7,1,7.0,23.877358
1,A Ashish Reddy,AB Dinda,9,0,4.5,30.852941
2,A Ashish Reddy,AD Mathews,25,0,4.5,40.481481
3,A Ashish Reddy,AD Russell,4,1,4.0,23.385965
4,A Ashish Reddy,Anureet Singh,2,0,4.5,35.333333


In [502]:
df['performance_index_against_bowler'] = df['batting_average']/df['career_bowling_average']
df

,batter,bowler,batsman_runs,is_wicket,batting_average,career_bowling_average,performance_index_against_bowler
0,A Ashish Reddy,A Nehra,7,1,7.000000,23.877358,0.293165
1,A Ashish Reddy,AB Dinda,9,0,4.500000,30.852941,0.145853
2,A Ashish Reddy,AD Mathews,25,0,4.500000,40.481481,0.111162
3,A Ashish Reddy,AD Russell,4,1,4.000000,23.385965,0.171043
4,A Ashish Reddy,Anureet Singh,2,0,4.500000,35.333333,0.127358
...,...,...,...,...,...,...,...
27231,Z Khan,SL Malinga,1,0,1.769231,21.134146,0.083714
27232,Z Khan,SR Watson,3,0,1.769231,29.967033,0.059039
27233,Z Khan,Shakib Al Hasan,3,0,1.769231,30.064516,0.058848
27234,Z Khan,Sohail Tanvir,3,0,1.769231,12.500000,0.141538


In [503]:
avg_pi_batsmen = df.groupby('batter').agg({'performance_index_against_bowler':'mean'}).reset_index()

In [504]:
avg_pi_batsmen['performance_index_against_bowler'].describe()

count    504.000000
mean       0.220765
std        0.187739
min        0.000000
25%        0.084214
50%        0.161721
75%        0.317982
max        0.944751
Name: performance_index_against_bowler, dtype: float64

In [505]:
avg_pi_batsmen[avg_pi_batsmen['batter'] == 'MS Dhoni']  

,batter,performance_index_against_bowler
279,MS Dhoni,0.701506


In [506]:
avg_pi_batsmen[avg_pi_batsmen['performance_index_against_bowler'] == avg_pi_batsmen['performance_index_against_bowler'].max()]

,batter,performance_index_against_bowler
475,V Kohli,0.944751


In [507]:
avg_pi_batsmen.columns = ['player', 'average_batting_performance_index']

In [508]:
avg_pi_batsmen

,player,average_batting_performance_index
0,A Ashish Reddy,0.161916
1,A Badoni,0.283516
2,A Chopra,0.089891
3,A Choudhary,0.124782
4,A Flintoff,0.079263
...,...,...
499,YV Dhull,0.110913
500,YV Takawale,0.129781
501,Yashpal Singh,0.153219
502,Yuvraj Singh,0.488666


In [509]:
avg_pi_batsmen.to_csv('batsmen_performance_index.csv')

In [548]:
# career bowling average = runs/wickets
# career batting average = runs/dismissed

## Performance Index for Bowlers

In [511]:
#df = deliveries_df.groupby(by=['bowler', 'batter']).agg({'total_runs':'sum', 'is_wicket':'sum'}).reset_index()

# First, let's create a mask for valid dismissals (wickets that count for bowling average).
# I only need to use the dismissals that are valid for calculating bowling average. So, I'll first create a boolean array depicting which dismissals are valid
valid_dismissals = ~deliveries_df['dismissal_kind'].isin(['run out', 'retired hurt', 'retired out', 'obstructing the field'])

# Now calculate the bowling statistics with this condition
df = deliveries_df.groupby(by=['bowler', 'batter']).agg({
    'total_runs': 'sum',
    # Only count wickets where is_wicket=1 AND dismissal_kind is valid. We use '&', not 'and' since we want to perform element wise "AND" operation across the two pandas series (x == 1) and (valid_dismissals[x.index])
    'is_wicket': lambda x: ((x == 1) & (valid_dismissals[x.index])).sum()
}).reset_index()
df

,bowler,batter,total_runs,is_wicket
0,A Ashish Reddy,AB de Villiers,48,0
1,A Ashish Reddy,AC Thomas,10,0
2,A Ashish Reddy,AD Mathews,3,0
3,A Ashish Reddy,AD Russell,1,0
4,A Ashish Reddy,AL Menaria,9,0
...,...,...,...,...
27232,Z Khan,WA Mota,1,0
27233,Z Khan,WP Saha,39,0
27234,Z Khan,Y Venugopal Rao,9,0
27235,Z Khan,YK Pathan,35,1


In [512]:
df.dropna(inplace=True)
df

,bowler,batter,total_runs,is_wicket
0,A Ashish Reddy,AB de Villiers,48,0
1,A Ashish Reddy,AC Thomas,10,0
2,A Ashish Reddy,AD Mathews,3,0
3,A Ashish Reddy,AD Russell,1,0
4,A Ashish Reddy,AL Menaria,9,0
...,...,...,...,...
27232,Z Khan,WA Mota,1,0
27233,Z Khan,WP Saha,39,0
27234,Z Khan,Y Venugopal Rao,9,0
27235,Z Khan,YK Pathan,35,1


In [513]:
df['bowling_average'] = df['total_runs']/df['is_wicket']
df

,bowler,batter,total_runs,is_wicket,bowling_average
0,A Ashish Reddy,AB de Villiers,48,0,inf
1,A Ashish Reddy,AC Thomas,10,0,inf
2,A Ashish Reddy,AD Mathews,3,0,inf
3,A Ashish Reddy,AD Russell,1,0,inf
4,A Ashish Reddy,AL Menaria,9,0,inf
...,...,...,...,...,...
27232,Z Khan,WA Mota,1,0,inf
27233,Z Khan,WP Saha,39,0,inf
27234,Z Khan,Y Venugopal Rao,9,0,inf
27235,Z Khan,YK Pathan,35,1,35.0


In [514]:
temp_df = df[~np.isinf(df['bowling_average'])]

In [515]:
best_per_bowler = temp_df.groupby(by=['bowler']).agg({'bowling_average':'mean'}).reset_index()

In [516]:
best_per_bowler

,bowler,bowling_average
0,A Ashish Reddy,6.266667
1,A Badoni,0.000000
2,A Chandila,3.272727
3,A Choudhary,2.400000
4,A Flintoff,1.000000
...,...,...
440,Yash Dayal,11.200000
441,Yash Thakur,9.630435
442,Yudhvir Singh,1.750000
443,Yuvraj Singh,11.275862


In [517]:
mapping = dict(zip(best_per_bowler['bowler'], best_per_bowler['bowling_average']))

In [518]:
df['bowling_average'] = np.where(
    df['bowling_average'] == np.inf,
    df['bowler'].map(mapping),
    df['bowling_average']
)

In [519]:
df['bowling_average'].describe()

count    26519.000000
mean        12.449386
std         10.410877
min          0.000000
25%          6.000000
50%         11.000000
75%         16.948113
max        192.000000
Name: bowling_average, dtype: float64

In [520]:
df['bowling_average'].replace(0, 1, inplace=True)

/var/folders/mz/ywg60v213710yk8053f7h4k00000gn/T/ipykernel_86616/842387900.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bowling_average'].replace(0, 1, inplace=True)


In [521]:
mapping = dict(zip(batting['batter'], batting['career_batting_average']))
df['career_batting_average'] = df['batter'].map(mapping)

In [522]:
df

,bowler,batter,total_runs,is_wicket,bowling_average,career_batting_average
0,A Ashish Reddy,AB de Villiers,48,0,6.266667,41.384000
1,A Ashish Reddy,AC Thomas,10,0,6.266667,10.000000
2,A Ashish Reddy,AD Mathews,3,0,6.266667,23.354839
3,A Ashish Reddy,AD Russell,1,0,6.266667,29.619048
4,A Ashish Reddy,AL Menaria,9,0,6.266667,17.909091
...,...,...,...,...,...,...
27232,Z Khan,WA Mota,1,0,14.766818,11.200000
27233,Z Khan,WP Saha,39,0,14.766818,24.247934
27234,Z Khan,Y Venugopal Rao,9,0,14.766818,23.452381
27235,Z Khan,YK Pathan,35,1,35.000000,28.954955


In [523]:
df['performance_index_against_batter'] = df['career_batting_average']/df['bowling_average']

In [524]:
avg_pi_bowler = df.groupby('bowler').agg({'performance_index_against_batter':'mean'}).reset_index()

In [525]:
avg_pi_bowler['performance_index_against_batter'].describe()

count    444.000000
mean       7.195177
std        7.558797
min        1.187795
25%        3.254261
50%        4.641943
75%        7.039354
max       51.905589
Name: performance_index_against_batter, dtype: float64

In [526]:
avg_pi_bowler[avg_pi_bowler['performance_index_against_batter'] == avg_pi_bowler['performance_index_against_batter'].min()]

,bowler,performance_index_against_batter
455,V Viyaskanth,1.187795


In [527]:
avg_pi_bowler[avg_pi_bowler['performance_index_against_batter'] == avg_pi_bowler['performance_index_against_batter'].max()]

,bowler,performance_index_against_batter
46,Abdul Samad,51.905589


In [528]:
avg_pi_bowler.sort_values(by='performance_index_against_batter', ascending=False).head(20)

,bowler,performance_index_against_batter
46,Abdul Samad,51.905589
79,C Nanda,48.180360
221,L Ablish,42.926652
407,SS Cottrell,40.816449
287,NB Singh,36.198334
465,WA Mota,35.291801
451,V Kohli,35.160595
418,Shashank Singh,34.852037
81,C de Grandhomme,32.899972
202,KJ Abbott,32.557154


In [529]:
avg_pi_bowler.columns = ['player', 'average_bowling_performance_index']

In [530]:
avg_pi_bowler.to_csv('bowler_performance_index.csv')

## Dealing with auction dataset

In [531]:
auction_data = pd.read_csv('IPLPlayerAuctionData.csv')

In [532]:
auction_data

,Player,Role,Amount,Team,Year,Player Origin
0,Aaron Finch,Batsman,40000000,Sunrisers Hyderabad,2014.0,Overseas
1,Aaron Finch,Batsman,32000000,Mumbai Indians,2015.0,Overseas
2,Aaron Finch,Batsman,10000000,Gujarat Lions,2016.0,Overseas
3,Aaron Finch,Batsman,62000000,Kings XI Punjab,2018.0,Overseas
4,Aaron Finch,Batsman,44000000,Royal Challengers Bangalore,2020.0,Overseas
...,...,...,...,...,...,...
965,Yuzvendra Singh Chahal,Bowler,1000000,Royal Challengers Bangalore,2014.0,Indian
966,Yuzvendra Singh Chahal,Bowler,60000000,Royal Challengers Bangalore,2018.0,Indian
967,Zaheer Khan,Bowler,26000000,Mumbai Indians,2014.0,Indian
968,Zaheer Khan,Bowler,40000000,Delhi Daredevils,2015.0,Indian


### Getting info about unsold players and retained players

In [549]:
import requests
from bs4 import BeautifulSoup

def getTable():
        
    url = "https://www.iplt20.com/auction/2022"
    headers = {'User-Agent': 'Mozilla/5.0'}  # Set a user-agent to avoid being blocked

    response = requests.get(url, headers=headers)
    #soup = BeautifulSoup(self.driver.page_source, 'html.parser')
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the section and the div with class 'tab-content'
    section = soup.find('div', class_='tab-content')
    if section:
        tab_content = section.find('div', id='autab4-2022')
        if tab_content:
            with open('tab_content.txt', 'w', encoding='utf-8') as f:
                f.write(tab_content.prettify())
        else:
            print("No div with class 'tab-content' found.")
    else:
        print("No section with class 'auction-tab-section' found.")

In [550]:
import csv

def makeCSV():

    # Load the HTML content from the text file
    with open('tab_content.txt', 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Open a CSV file to write the extracted data
    with open('unsold_players.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Player Name', 'Nationality', 'Type', 'Base Price'])  # Write headers

        # Locate the table body containing player data
        tbody = soup.find('tbody', id='pointsdata')
        
        if tbody:
            # Iterate through each row in the table body
            for row in tbody.find_all('tr'):
                # Extract player name, nationality, type, and base price
                player_name = row.find_all('td')[0].get_text(strip=True)
                nationality = row.find_all('td')[1].get_text(strip=True)
                player_type = row.find_all('td')[2].get_text(strip=True)
                base_price = row.find_all('td')[3].get_text(strip=True)

                # Write the extracted data to the CSV file
                writer.writerow([player_name, nationality, player_type, base_price])

    print("Data extracted and saved to unsold_players.csv")

In [551]:
getTable()
makeCSV()

Data extracted and saved to unsold_players.csv


In [552]:
unsoldPlayers_df = pd.read_csv("unsold_players.csv")
unsoldPlayers_df

,Player Name,Nationality,Type,Base Price
0,Suresh Raina,Indian,Batsman,"2,00,00,000"
1,Steve Smith,Overseas,Batsman,"2,00,00,000"
2,Shakib Al Hasan,Overseas,All-Rounder,"2,00,00,000"
3,Adil Rashid,Overseas,Bowler,"2,00,00,000"
4,Imran Tahir,Overseas,Bowler,"2,00,00,000"
...,...,...,...,...
391,K Nitish Reddy,Indian,Batsman,"20,00,000"
392,Hardik Tamore,Indian,Wicket Keeper,"20,00,000"
393,Mihir Hirwani,Indian,Bowler,"20,00,000"
394,Sairaj Patil,Indian,All-Rounder,"20,00,000"


### Getting info about RCB players retained in 2021

In [622]:
retainedPlayers = pd.read_csv("retained_players.csv")
retainedPlayers.drop(columns=['Unnamed: 0'], inplace=True)
retainedPlayers

,Player,Player Origin,Role
0,Virat Kohli,Indian,Batsman
1,Srikar Bharat,Indian,Wicket Keeper
2,Tim David,Overseas,Batsman
3,AB de Villiers,Overseas,Wicket Keeper
4,Mohammed Azharuddeen,Indian,Wicket Keeper
5,Devdutt Padikkal,Indian,Batsman
6,Rajat Patidar,Indian,Batsman
7,Sachin Baby,Indian,Batsman
8,Finn Allen,Overseas,Batsman
9,Josh Philippe,Overseas,Wicket Keeper


In [553]:
auction_data

,Player,Role,Amount,Team,Year,Player Origin
0,Aaron Finch,Batsman,40000000,Sunrisers Hyderabad,2014.0,Overseas
1,Aaron Finch,Batsman,32000000,Mumbai Indians,2015.0,Overseas
2,Aaron Finch,Batsman,10000000,Gujarat Lions,2016.0,Overseas
3,Aaron Finch,Batsman,62000000,Kings XI Punjab,2018.0,Overseas
4,Aaron Finch,Batsman,44000000,Royal Challengers Bangalore,2020.0,Overseas
...,...,...,...,...,...,...
965,Yuzvendra Singh Chahal,Bowler,1000000,Royal Challengers Bangalore,2014.0,Indian
966,Yuzvendra Singh Chahal,Bowler,60000000,Royal Challengers Bangalore,2018.0,Indian
967,Zaheer Khan,Bowler,26000000,Mumbai Indians,2014.0,Indian
968,Zaheer Khan,Bowler,40000000,Delhi Daredevils,2015.0,Indian


In [564]:
#Renaming columns of unsoldPlayers_df
unsoldPlayers_df.rename(columns={'Player Name': 'Player', 'Nationality': 'Player Origin', 'Type': 'Role', 'Base Price': 'Amount'}, inplace=True)
unsoldPlayers_df['Amount'] = unsoldPlayers_df['Amount'].str.replace(',', '').astype('int64')      #Converting Indian numeral form to normal form
unsoldPlayers_df

,Player,Player Origin,Role,Amount
0,Suresh Raina,Indian,Batsman,20000000
1,Steve Smith,Overseas,Batsman,20000000
2,Shakib Al Hasan,Overseas,All-Rounder,20000000
3,Adil Rashid,Overseas,Bowler,20000000
4,Imran Tahir,Overseas,Bowler,20000000
...,...,...,...,...
391,K Nitish Reddy,Indian,Batsman,2000000
392,Hardik Tamore,Indian,Wicket Keeper,2000000
393,Mihir Hirwani,Indian,Bowler,2000000
394,Sairaj Patil,Indian,All-Rounder,2000000


In [533]:
avg_pi_batsmen

,player,average_batting_performance_index
0,A Ashish Reddy,0.161916
1,A Badoni,0.283516
2,A Chopra,0.089891
3,A Choudhary,0.124782
4,A Flintoff,0.079263
...,...,...
499,YV Dhull,0.110913
500,YV Takawale,0.129781
501,Yashpal Singh,0.153219
502,Yuvraj Singh,0.488666


In [534]:
players = pd.merge(avg_pi_batsmen, avg_pi_bowler, on='player', how='outer')


In [535]:
players.fillna(0, inplace=True)

In [536]:
players

,player,average_batting_performance_index,average_bowling_performance_index
0,A Ashish Reddy,0.161916,5.846508
1,A Badoni,0.283516,30.832597
2,A Chandila,0.000000,9.879696
3,A Chopra,0.089891,0.000000
4,A Choudhary,0.124782,13.102461
...,...,...,...
675,Yash Thakur,0.000000,4.438647
676,Yashpal Singh,0.153219,0.000000
677,Yudhvir Singh,0.000000,16.099232
678,Yuvraj Singh,0.488666,3.494895


In [537]:
players.to_csv('player_performance_index.csv')

In [538]:
players = pd.read_csv('player_performance_index.csv', index_col = 0)

In [539]:
players

,player,average_batting_performance_index,average_bowling_performance_index
0,A Ashish Reddy,0.161916,5.846508
1,A Badoni,0.283516,30.832597
2,A Chandila,0.000000,9.879696
3,A Chopra,0.089891,0.000000
4,A Choudhary,0.124782,13.102461
...,...,...,...
675,Yash Thakur,0.000000,4.438647
676,Yashpal Singh,0.153219,0.000000
677,Yudhvir Singh,0.000000,16.099232
678,Yuvraj Singh,0.488666,3.494895


In [540]:
import requests
from bs4 import BeautifulSoup

# Step 1: Make a request to Google
search_query = "A Ashish Reddy"
url = f"https://www.google.com/search?q={search_query}"
headers = {'User-Agent': 'Mozilla/5.0'}  # Set a user-agent to avoid being blocked

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

results = soup.find_all('div')  

# Print out the results
for result in results:
    if 'www.espncricinfo.com' in result.get_text() and 'Profile' in result.get_text():
        name = result.get_text().split('Profile')[0].strip()


In [541]:
name

'Ashish Reddy'

In [542]:
def get_full_name(search_query):
    name = search_query
    # Step 1: Make a request to Google
    # search_query = "A Ashish Reddy"
    url = f"https://www.google.com/search?q={search_query}"
    headers = {'User-Agent': 'Mozilla/5.0'}  # Set a user-agent to avoid being blocked

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find_all('div')  

    # Print out the results
    for result in results:
        if 'www.espncricinfo.com' in result.get_text() and 'Profile' in result.get_text():
            name = result.get_text().split('Profile')[0].strip()
    
    return name


In [48]:
import time

# Assuming players is already defined and has a 'player' attribute
name_map = {}
for player in players['player'].to_list():
    print(player)
    full_name = get_full_name(player)
    print(full_name)
    name_map[player] = full_name
    time.sleep(1.5)

# Uncomment the following line if you want to see the final map
# print(name_map)


A Ashish Reddy
Ashish Reddy
A Badoni
Ayush Badoni
A Chandila
Ajit Chandila
A Chopra
Agni Chopra
A Choudhary
Aniket Choudhary
A Dananjaya
Akila Dananjaya
A Flintoff
Andrew Flintoff
A Kamboj
Anshul Kamboj
A Kumble
Anil Kumble
A Manohar
Abhinav Manohar
A Mishra
Amit Mishra
A Mithun
Abhimanyu Mithun
A Mukund
Abhinav Mukund
A Nehra
Ashish Nehra
A Nel
Andre Nel
A Nortje
Anrich Nortje
A Raghuvanshi
Angkrish Raghuvanshi
A Singh
Anmolpreet Singh
A Symonds
Andrew Symonds
A Uniyal
Amit Uniyal
A Zampa
Adam Zampa
AA Bilakhia
Azhar Bilakhia
AA Chavan
Ankeet Chavan
AA Jhunjhunwala
Abhishek Jhunjhunwala
AA Noffke
Ashley Noffke
AB Agarkar
Ajit Agarkar
AB Barath
Adrian Barath
AB Dinda
Ashok Dinda
AB McDonald
Andrew McDonald
AB de Villiers
AB de Villiers
AC Blizzard
Aiden Blizzard
AC Gilchrist
Adam Gilchrist
AC Thomas
AC Thomas
AC Voges
Adam Voges
AD Hales
Alex Hales
AD Mascarenhas
AllImagesNewsVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAl

ConnectionError: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=CA%20Lynn (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x72034d4c2190>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [50]:
name_map

{'A Ashish Reddy': 'Ashish Reddy',
 'A Badoni': 'Ayush Badoni',
 'A Chandila': 'Ajit Chandila',
 'A Chopra': 'Agni Chopra',
 'A Choudhary': 'Aniket Choudhary',
 'A Dananjaya': 'Akila Dananjaya',
 'A Flintoff': 'Andrew Flintoff',
 'A Kamboj': 'Anshul Kamboj',
 'A Kumble': 'Anil Kumble',
 'A Manohar': 'Abhinav Manohar',
 'A Mishra': 'Amit Mishra',
 'A Mithun': 'Abhimanyu Mithun',
 'A Mukund': 'Abhinav Mukund',
 'A Nehra': 'Ashish Nehra',
 'A Nel': 'Andre Nel',
 'A Nortje': 'Anrich Nortje',
 'A Raghuvanshi': 'Angkrish Raghuvanshi',
 'A Singh': 'Anmolpreet Singh',
 'A Symonds': 'Andrew Symonds',
 'A Uniyal': 'Amit Uniyal',
 'A Zampa': 'Adam Zampa',
 'AA Bilakhia': 'Azhar Bilakhia',
 'AA Chavan': 'Ankeet Chavan',
 'AA Jhunjhunwala': 'Abhishek Jhunjhunwala',
 'AA Noffke': 'Ashley Noffke',
 'AB Agarkar': 'Ajit Agarkar',
 'AB Barath': 'Adrian Barath',
 'AB Dinda': 'Ashok Dinda',
 'AB McDonald': 'Andrew McDonald',
 'AB de Villiers': 'AB de Villiers',
 'AC Blizzard': 'Aiden Blizzard',
 'AC Gilch

In [54]:
index = players['player'].to_list().index('Gagandeep Singh')
for player in players['player'].to_list()[index:]:
    full_name = get_full_name(player)
    print(full_name)
    name_map[player] = full_name
    time.sleep(1.5)


Gagandeep Singh
Gurkeerat Singh Mann
Gurnoor Brar
Heinrich Klaasen
Himanshu Sharma
Harry Brook
AllImagesNewsVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimRassie van der DussenSouth African cricketerView allHendrik Erasmus "Rassie" van der Dussen is a South African professional cricketer who represents the South Africa national cricket team and plays for Gauteng in domestic cricket. In the 2018 South African Cricket Annual, he was named as one of the... WikipediaBorn: 7 February 1989 (age 35 years), Pretoria, South AfricaSpouse: Lara van der DussenCurrent teams: South Africa national cricket team, MI Cape Town (Batter), Vancouver Knights (Batter) and moreDates joined: 2024 (Lahore Qalandars), 2023 (MI Cape Town, Islamabad United), February 2022 (Rajasthan Royals) and moreBatting: Right-handedBowling: Right arm leg breakLast ODI: 4 October 2024 v IrelandShow moreShow lessRassie van der Dussen - C

In [543]:
players['player'][0]

'A Ashish Reddy'

In [544]:
name_map

NameError: name 'name_map' is not defined

In [57]:
players.replace({'player':name_map}, inplace=True)      #Replace the 'player' column with the values of 'player' from the name_map dictionary

In [58]:
players.to_csv('player_performance_index.csv')

In [81]:
auction_data = pd.read_csv('IPLPlayerAuctionData.csv')

In [82]:
auction_data

,Player,Role,Amount,Team,Year,Player Origin
0,Aaron Finch,Batsman,40000000,Sunrisers Hyderabad,2014.0,Overseas
1,Aaron Finch,Batsman,32000000,Mumbai Indians,2015.0,Overseas
2,Aaron Finch,Batsman,10000000,Gujarat Lions,2016.0,Overseas
3,Aaron Finch,Batsman,62000000,Kings XI Punjab,2018.0,Overseas
4,Aaron Finch,Batsman,44000000,Royal Challengers Bangalore,2020.0,Overseas
...,...,...,...,...,...,...
965,Yuzvendra Singh Chahal,Bowler,1000000,Royal Challengers Bangalore,2014.0,Indian
966,Yuzvendra Singh Chahal,Bowler,60000000,Royal Challengers Bangalore,2018.0,Indian
967,Zaheer Khan,Bowler,26000000,Mumbai Indians,2014.0,Indian
968,Zaheer Khan,Bowler,40000000,Delhi Daredevils,2015.0,Indian


In [582]:
unsoldPlayers_df['Year'] = 2022.0       #Adding an year column for consistent merging of unsold players and other players of the auction
unsoldPlayers_df

,Player,Player Origin,Role,Amount,Year
0,Suresh Raina,Indian,Batsman,20000000,2022.0
1,Steve Smith,Overseas,Batsman,20000000,2022.0
2,Shakib Al Hasan,Overseas,All-Rounder,20000000,2022.0
3,Adil Rashid,Overseas,Bowler,20000000,2022.0
4,Imran Tahir,Overseas,Bowler,20000000,2022.0
...,...,...,...,...,...
391,K Nitish Reddy,Indian,Batsman,2000000,2022.0
392,Hardik Tamore,Indian,Wicket Keeper,2000000,2022.0
393,Mihir Hirwani,Indian,Bowler,2000000,2022.0
394,Sairaj Patil,Indian,All-Rounder,2000000,2022.0


In [577]:
latest_records = auction_data.loc[auction_data.groupby('Player')['Year'].idxmax()]
#First we group the dataFrame by the player names, And then find the indices of the rows that have the highest value for the 'Year' column
    #Then, we use loc to only select those columns that have the highest value of 'Year' for each player

In [583]:
latest_records
#latest_records.dtypes

,Player,Role,Amount,Team,Year,Player Origin
4,Aaron Finch,Batsman,44000000,Royal Challengers Bangalore,2020.0,Overseas
5,Abdul Samad,All-Rounder,2000000,Sunrisers Hyderabad,2020.0,Indian
6,Abhijeet Tomar,Batsman,4000000,Kolkata Knight Riders,2022.0,Indian
8,Abhimanyu Mithun,Bowler,3000000,Sunrisers Hyderabad,2016.0,Indian
9,Abhinav Sadarangani,Batsman,26000000,Gujarat Titans,2022.0,Indian
...,...,...,...,...,...,...
963,Yuvraj Singh,All-Rounder,10000000,Mumbai Indians,2019.0,Indian
964,Yuzvendra Chahal,Bowler,65000000,Rajasthan Royals,2022.0,Indian
966,Yuzvendra Singh Chahal,Bowler,60000000,Royal Challengers Bangalore,2018.0,Indian
968,Zaheer Khan,Bowler,40000000,Delhi Daredevils,2015.0,Indian


In [585]:
import time
#Extracting standardised names of unsold players from Google
unsoldPlayers_name_map = {}

for player in unsoldPlayers_df['Player'].to_list():
    full_name = get_full_name(player)
    print(full_name)
    unsoldPlayers_name_map[player] = full_name
    time.sleep(1.5)

Suresh Raina
Steven Smith
Shakib Al Hasan
Adil Rashid
Imran Tahir
Mujeeb Zadran
Adam Zampa
James Vince
Marchant de Lange
Saqib Mahmood
Ashton Agar
Craig Overton
Amit Mishra
Aaron Finch
Dawid Malan
Eoin Morgan
Ishant Sharma
Chris Lynn
Usman Khawaja
Lewis Gregory
AllNewsImagesVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimKane Richardson - Cricket Player Australia - ESPNcricinfowww.espncricinfo.com › cricketers › kane-richardson-272262A pace bowler who has often been on the fringes of Australia's ODI and T20 teams, Kane Richardson is know for his skills at the death and has an excellent ...Stats · Matches · News · VideosKane Richardson
Colin Munro
AllNewsImagesVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimMarnus Labuschagne - Cricket Player Australia - ESPNcricinfowww.espncricinfo.com › cricketers › marnus

KeyboardInterrupt: 

In [615]:
#Extracting standardised names of retained players from Google
retainedPlayers_name_map = {}

for player in retainedPlayers['Player'].to_list():
    full_name = get_full_name(player)
    print(full_name)
    retainedPlayers_name_map[player] = full_name
    time.sleep(1.5)

Virat Kohli
Srikar Bharat
Tim David
AB de Villiers
Mohammed Azharuddeen
Devdutt Padikkal
Rajat Patidar
Sachin Baby
Finn Allen
Josh Philippe
Dan Christian
Pavan Deshpande
 George Canton
Glenn Maxwell
Suyash Prabhudessai
Shahbaz Ahmed
AllNewsImagesVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimScott Kuggeleijn - Cricket Player New Zealand - ESPNcricinfowww.espncricinfo.com › cricketers › scott-kuggeleijn-539548Read about Scott Kuggeleijn cricket player from New Zealand profile, stats, rankings, records, videos, photos at ESPNcricinfo.Scott Kuggeleijn - Wikipediaen.wikipedia.org › wiki › Scott_KuggeleijnScott Christopher Kuggeleijn (born 3 January 1992) is a New Zealand international cricketer. He plays first-class cricket for Northern Districts.Scott Kuggeleijn
Daniel Sams
Washington Sundar
Akash Deep
Yuzvendra Chahal
Kyle Jamieson
Mohammed Siraj
Harshal Patel
Navdeep Saini
Dushmantha Chameera
All

In [85]:
for player in latest_records['Player'].to_list():
    full_name = get_full_name(player)
    print(full_name)
    name_map[player] = full_name
    time.sleep(1.5)


Aaron Finch
Abdul Samad
Abhijeet Tomar
Abhimanyu Mithun
Abhinav Manohar
Abhishek Nayar
Abhishek Sharma
Abu Nechim
Adam Milne
Adam Zampa
Aditya Garhwal
Aditya Tare
Agnivesh Ayachi
Aiden Blizzard
Aiden Markram
Ajantha Mendis
Ajinkya Rahane
Akash Deep
Akash Singh
Akhil Herwadkar
Akila Dananjaya
Akila Dananjaya
Axar Patel
Akshay Karnewar
Akshay Wakhare
Akshdeep Nath
AllNewsImagesVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimAlbie Morkel - Cricket Player South Africa - ESPNcricinfowww.espncricinfo.com › cricketers › albie-morkel-46248A medium-fast bowler and hard-hitting batsman, he was likened to Lance Klusener but his domestic dominance failed to translate to the international level.Matches · Photos · Stats · RecordsAlbie Morkel (@albiemorkel81) • Instagram photos and videoswww.instagram.com › albiemorkel81New coach alert Man of the Series in the Proteas' ODI series victory over Australia in 2009.

In [616]:
unsoldPlayers_name_map
retainedPlayers_name_map

{'Virat Kohli': 'Virat Kohli',
 ' Srikar Bharat': 'Srikar Bharat',
 ' Tim David': 'Tim David',
 ' AB de Villiers': 'AB de Villiers',
 ' Mohammed Azharuddeen': 'Mohammed Azharuddeen',
 ' Devdutt Padikkal': 'Devdutt Padikkal',
 ' Rajat Patidar': 'Rajat Patidar',
 ' Sachin Baby': 'Sachin Baby',
 ' Finn Allen': 'Finn Allen',
 ' Josh Philippe': 'Josh Philippe',
 ' Dan Christian': 'Dan Christian',
 ' Pavan Deshpande': 'Pavan Deshpande',
 ' George Canton': ' George Canton',
 ' Glenn Maxwell': 'Glenn Maxwell',
 ' Suyash Prabhudessai': 'Suyash Prabhudessai',
 ' Shahbaz Ahmed': 'Shahbaz Ahmed',
 ' Scott Kuggeleijn': 'AllNewsImagesVideos Maps Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimScott Kuggeleijn - Cricket Player New Zealand - ESPNcricinfowww.espncricinfo.com › cricketers › scott-kuggeleijn-539548Read about Scott Kuggeleijn cricket player from New Zealand profile, stats, rankings, records, videos, photos at E

In [623]:
latest_records.replace({'Player':name_map}, inplace=True)
unsoldPlayers_df.replace({'Player': unsoldPlayers_name_map}, inplace=True)
retainedPlayers.replace({'Player': retainedPlayers_name_map}, inplace=True)

In [624]:
latest_records.to_csv('latest_records.csv')
unsoldPlayers_df.to_csv('unsoldPlayers_latest_records.csv')
retainedPlayers.to_csv('retained_players_final.csv')

In [595]:
latest_records = pd.read_csv('latest_records.csv', index_col=0)
unsoldPlayers_latest_records_df = pd.read_csv('unsoldPlayers_latest_records.csv', index_col=0)
unsoldPlayers_latest_records_df

,Player,Player Origin,Role,Amount,Year
0,Suresh Raina,Indian,Batsman,20000000,2022.0
1,Steve Smith,Overseas,Batsman,20000000,2022.0
2,Shakib Al Hasan,Overseas,All-Rounder,20000000,2022.0
3,Adil Rashid,Overseas,Bowler,20000000,2022.0
4,Imran Tahir,Overseas,Bowler,20000000,2022.0
...,...,...,...,...,...
391,K Nitish Reddy,Indian,Batsman,2000000,2022.0
392,Hardik Tamore,Indian,Wicket Keeper,2000000,2022.0
393,Mihir Hirwani,Indian,Bowler,2000000,2022.0
394,Sairaj Patil,Indian,All-Rounder,2000000,2022.0


In [86]:
players = pd.read_csv('player_performance_index.csv', index_col=0)
players.columns = ['Player', 'average_batting_performance_index',
       'average_bowling_performance_index']

In [88]:
players

,Player,average_batting_performance_index,average_bowling_performance_index
0,A Ashish Reddy,0.194312,6.405304
1,A Badoni,0.322596,30.832597
2,A Chandila,0.000000,9.879696
3,A Chopra,0.116028,0.000000
4,A Choudhary,0.144900,13.102461
...,...,...,...
675,Yash Thakur,0.000000,4.654120
676,Yashpal Singh,0.173776,0.000000
677,Yudhvir Singh,0.000000,16.099232
678,Yuvraj Singh,0.563578,3.284710


In [596]:
latest_records.drop(columns=['Team', 'Year', 'Team'], inplace=True)
unsoldPlayers_latest_records_df.drop(columns=['Year'], inplace=True)

In [92]:
latest_records

,Player,Role,Amount,Player Origin
4,Aaron Finch,Batsman,44000000,Overseas
5,Abdul Samad,All-Rounder,2000000,Indian
6,Abhijeet Tomar,Batsman,4000000,Indian
8,Abhimanyu Mithun,Bowler,3000000,Indian
9,Abhinav Manohar,Batsman,26000000,Indian
...,...,...,...,...
964,Yuzvendra Chahal,Bowler,65000000,Indian
966,Yuzvendra Chahal,Bowler,60000000,Indian
968,Zaheer Khan,Bowler,40000000,Indian
969,Zahir Khan,Bowler,6000000,Indian


In [599]:
unsoldPlayers_latest_records = unsoldPlayers_latest_records_df
unsoldPlayers_latest_records

,Player,Player Origin,Role,Amount
0,Suresh Raina,Indian,Batsman,20000000
1,Steve Smith,Overseas,Batsman,20000000
2,Shakib Al Hasan,Overseas,All-Rounder,20000000
3,Adil Rashid,Overseas,Bowler,20000000
4,Imran Tahir,Overseas,Bowler,20000000
...,...,...,...,...
391,K Nitish Reddy,Indian,Batsman,2000000
392,Hardik Tamore,Indian,Wicket Keeper,2000000
393,Mihir Hirwani,Indian,Bowler,2000000
394,Sairaj Patil,Indian,All-Rounder,2000000


In [621]:
retainedPlayers_latest_records = retainedPlayers
retainedPlayers_latest_records

,Player,Player Origin,Role,Auction_status
0,Virat Kohli,Indian,Batsman,Retained
1,Srikar Bharat,Indian,Wicket Keeper,Retained
2,Tim David,Overseas,Batsman,Retained
3,AB de Villiers,Overseas,Wicket Keeper,Retained
4,Mohammed Azharuddeen,Indian,Wicket Keeper,Retained
5,Devdutt Padikkal,Indian,Batsman,Retained
6,Rajat Patidar,Indian,Batsman,Retained
7,Sachin Baby,Indian,Batsman,Retained
8,Finn Allen,Overseas,Batsman,Retained
9,Josh Philippe,Overseas,Wicket Keeper,Retained


In [625]:
latest_records['Player'] = latest_records['Player'].drop_duplicates(keep='last')    #Duplicates exist because the same player has a different auction prices across different years
                                                                                        #So, we make sure to keep only the last (latest) auction price of every player

latest_records['Auction_status'] = 'Sold'   #Adding an extra auction status column to differentiate between 'Sold', 'Unsold', and 'Retained' players later
unsoldPlayers_latest_records['Auction_status'] = 'Unsold'
retainedPlayers_latest_records['Auction_status'] = 'Retained'


In [627]:
#Merging auction and latest record dataset
latest_records_soldPlusRetained = pd.merge(latest_records, retainedPlayers_latest_records, on='Player', how='outer')

#Concatenating sold and unsold players
latest_records = pd.concat([latest_records_soldPlusRetained, unsoldPlayers_latest_records])
latest_records

,Player,Role_x,Amount,Player Origin_x,Auction_status_x,Player Origin_y,Role_y,Auction_status_y,Player Origin,Role,Auction_status
0,George Canton,NaN,NaN,NaN,NaN,Overseas,All-Rounder,Retained,NaN,NaN,NaN
1,AB de Villiers,NaN,NaN,NaN,NaN,Overseas,Wicket Keeper,Retained,NaN,NaN,NaN
2,Aaqib Khan,Bowler,2000000.0,Indian,Unsold,NaN,NaN,NaN,NaN,NaN,NaN
3,Aaron Hardie,All-Rounder,2000000.0,Overseas,Unsold,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaron Finch,Batsman,44000000.0,Overseas,Sold,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
391,K Nitish Reddy,NaN,2000000.0,NaN,NaN,NaN,NaN,NaN,Indian,Batsman,Unsold
392,Hardik Tamore,NaN,2000000.0,NaN,NaN,NaN,NaN,NaN,Indian,Wicket Keeper,Unsold
393,Mihir Hirwani,NaN,2000000.0,NaN,NaN,NaN,NaN,NaN,Indian,Bowler,Unsold
394,Sairaj Patil,NaN,2000000.0,NaN,NaN,NaN,NaN,NaN,Indian,All-Rounder,Unsold


In [628]:
final_df = pd.merge(players, latest_records, on='Player', how='left')       
#We are using left merge since we want to preserve all the players from the 'players' dataset, but only enrich them with additional data from the 'latest records' dataset

KeyError: 'Player'

In [95]:
final_df.isna().sum()

Player                                 0
average_batting_performance_index      0
average_bowling_performance_index      0
Role                                 613
Amount                               613
Player Origin                        613
dtype: int64

In [98]:
final_df

,Player,average_batting_performance_index,average_bowling_performance_index,Role,Amount,Player Origin
0,A Ashish Reddy,0.194312,6.405304,NaN,NaN,NaN
1,A Badoni,0.322596,30.832597,NaN,NaN,NaN
2,A Chandila,0.000000,9.879696,NaN,NaN,NaN
3,A Chopra,0.116028,0.000000,NaN,NaN,NaN
4,A Choudhary,0.144900,13.102461,NaN,NaN,NaN
...,...,...,...,...,...,...
675,Yash Thakur,0.000000,4.654120,NaN,NaN,NaN
676,Yashpal Singh,0.173776,0.000000,NaN,NaN,NaN
677,Yudhvir Singh,0.000000,16.099232,All-Rounder,2000000.0,Indian
678,Yuvraj Singh,0.563578,3.284710,All-Rounder,10000000.0,Indian


In [99]:
final_df[final_df['Player'] == 'Virat Kohli']

,Player,average_batting_performance_index,average_bowling_performance_index,Role,Amount,Player Origin


In [97]:
final_df['Player'].is_unique

True

In [100]:
len(final_df['Player'].unique())

680

In [101]:
final_df.dropna(inplace=True)

In [102]:
final_df.reset_index(inplace=True)

In [103]:
final_df.drop(columns=['index'], inplace=True)

In [104]:
final_df

,Player,average_batting_performance_index,average_bowling_performance_index,Role,Amount,Player Origin
0,Abdul Samad,0.315572,51.905589,All-Rounder,2000000.0,Indian
1,Abhishek Sharma,0.491760,5.288493,All-Rounder,65000000.0,Indian
2,Akash Deep,0.000000,3.260651,Bowler,2000000.0,Indian
3,Akash Singh,0.000000,5.655292,Bowler,2000000.0,Indian
4,Ankit Sharma,0.143833,2.250009,All-Rounder,2000000.0,Indian
...,...,...,...,...,...,...
62,Vishnu Vinod,0.123803,0.000000,Wicket Keeper,5000000.0,Indian
63,Washington Sundar,0.277149,2.824106,All-Rounder,87500000.0,Indian
64,Yash Dayal,0.000000,3.965339,Bowler,32000000.0,Indian
65,Yudhvir Singh,0.000000,16.099232,All-Rounder,2000000.0,Indian


In [105]:
final_df.to_csv('final.csv')